# RL Test Bench

In [1]:
import sys
pwd = "P:/dsv/DAMII/Practical Project/rl_stock_agents"
sys.path.append(pwd)

import os
import cleandata
import numpy as np
import matplotlib.pyplot as plt
from agents import Discrete_QtabAgent, Discrete_Random_Agent, Discrete_Buy_Hold_Agent
from agentperform import agent_stock_performance, aggregate_trade_performance, update_trade_df, graph_input
import logging
import pandas as pd

# Output Location
main_path = "P:/dsv/DAMII/Practical Project/rl_stock_agents/test_bench"
test_dir = '/_project_test/'
save_path = main_path + test_dir

## Check if the directory exists; if not, create it
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Logger Setup (Do not change)
logging.basicConfig(level=logging.INFO, filemode='w')
## Test Logger
test_logger = logging.getLogger('Test_logger')
test_handler = logging.FileHandler(save_path +'/test.log')
test_handler.setLevel(logging.INFO)
test_logger.addHandler(test_handler)

## History Logger
history_logger = logging.getLogger('History_logger')
history_handler = logging.FileHandler(save_path + '/history.log')
history_handler.setLevel(logging.INFO)
history_logger.addHandler(history_handler)

## Model Logger
model_compare_logger = logging.getLogger('Model_compare_logger')
model_compare_handler = logging.FileHandler(save_path + '/model_compare.log')
model_compare_handler.setLevel(logging.INFO)
model_compare_logger.addHandler(model_compare_handler)

## Workbench Configuration

In [2]:
# Input Data Location, File Name, Stock name for labels
import_path = "P:/dsv/DAMII/Practical Project/rl_stock_agents/input_data"

stock_list =["amzn_daily.csv", "appl_daily.csv","tsla_daily.csv","f_daily.csv",
             "pfe_daily.csv", "coke_daily.csv","brk_daily.csv","nee_daily.csv",
             "jnj_daily.csv","pg_daily.csv"]

stock_name_list = ['AMZN','APPL','TSLA','F','PFE','COKE','BRK','NEE','JNJ','PG']

# Configuration for Grpahic Output
save_raw_graphic_option = False
output_raw_graphic_option = False
save_agg_graphic_option = True
output_agg_graphic_option = False
save_input_graphic_option = True
output_input_graphic_option = False

# Common Model Parameters 
training_range = [[0,1000],[1250,2250],[0,2000]]
test_range = [[1000,1250],[2250,2518],[2000,2518]]
epochs = 500
num_of_tests = 20
metrics = ['n_trades','win_percentage','cumulative_return','sortino','max_drawdown','sharpe']
## See agentperform.py -> results dictionary for options
## current setting "n_trades" has its ranks based on min (manual change in
## prob._valuate.py line 92. Need to create feature to avoid this manual change)
rank_order = ['min','max','max','max','min','max']


# Agent Declariations 

agent_classes = {'Qtab_[2,2,2,2,2]_raw': Discrete_QtabAgent,
                 'Qtab_[10,10,10,10,10]_raw': Discrete_QtabAgent,
                 'Qtab_[10,10,10,10,10]_diff': Discrete_QtabAgent,
                 'Qtab_[10,10,10,10,10]_ma': Discrete_QtabAgent,
                 'Qtab_[100,100,100,100,100]_raw': Discrete_QtabAgent,
                 'Qtab_[100,100,100,100,100]_diff': Discrete_QtabAgent,
                 'Qtab_[100,100,100,100,100]_ma': Discrete_QtabAgent,
                 'Random_Agent': Discrete_Random_Agent,
                 'Buy_Hold_Agent':Discrete_Buy_Hold_Agent}

agent_list = list(agent_classes.keys())

no_train_agents = ['Random_Agent','Buy_Hold_Agent']  


agent_params = {
    'Qtab_[2,2,2,2,2]_raw':{
        'bins_per_feature': [2,2,2,2,2],
        'bin_padding': 0.25,
        'ohlcv_behaviour': 'raw',
        'training_idxs': training_range,
        'num_training_episodes': epochs,
        'testing_idxs': test_range,
        'epsilon':  0.1,
        'alpha':  0.15,
        'gamma':  0.95
    },

        'Qtab_[10,10,10,10,10]_raw':{
            'bins_per_feature': [10,10,10,10,10],
            'bin_padding': 0.25,
            'ohlcv_behaviour': 'raw',
            'training_idxs': training_range,
            'num_training_episodes': epochs,
            'testing_idxs': test_range,
            'epsilon':  0.1,
            'alpha':  0.15,
            'gamma':  0.95
    },

        'Qtab_[10,10,10,10,10]_diff':{
            'bins_per_feature': [10,10,10,10,10],
            'bin_padding': 0.25,
            'ohlcv_behaviour': 'diff',
            'training_idxs': training_range,
            'num_training_episodes': epochs,
            'testing_idxs': test_range,
            'epsilon':  0.1,
            'alpha':  0.15,
            'gamma':  0.95
    },
            'Qtab_[10,10,10,10,10]_ma':{
            'bins_per_feature': [10,10,10,10,10],
            'bin_padding': 0.25,
            'ohlcv_behaviour': '1_3_4_ma__v',
            'training_idxs': training_range,
            'num_training_episodes': epochs,
            'testing_idxs': test_range,
            'epsilon':  0.1,
            'alpha':  0.15,
            'gamma':  0.95
    },

        'Qtab_[100,100,100,100,100]_raw':{
            'bins_per_feature': [100,100,100,100,100],
            'bin_padding': 0.25,
            'ohlcv_behaviour': 'raw',
            'training_idxs': training_range,
            'num_training_episodes': epochs,
            'testing_idxs': test_range,
            'epsilon':  0.1,
            'alpha':  0.15,
            'gamma':  0.95
    },

        'Qtab_[100,100,100,100,100]_diff':{
            'bins_per_feature': [100,100,100,100,100],
            'bin_padding': 0.25,
            'ohlcv_behaviour': 'diff',
            'training_idxs': training_range,
            'num_training_episodes': epochs,
            'testing_idxs': test_range,
            'epsilon':  0.1,
            'alpha':  0.15,
            'gamma':  0.95
    },
            'Qtab_[100,100,100,100,100]_ma':{
            'bins_per_feature': [100,100,100,100,100],
            'bin_padding': 0.25,
            'ohlcv_behaviour': '1_3_4_ma__v',
            'training_idxs': training_range,
            'num_training_episodes': epochs,
            'testing_idxs': test_range,
            'epsilon':  0.1,
            'alpha':  0.15,
            'gamma':  0.95
    },


    'Random_Agent':{
        'bins_per_feature': [2,2,2,2,2],
        'bin_padding': 0.25,     
        'training_idxs': training_range,
        'testing_idxs': test_range,
    },
        'Buy_Hold_Agent':{
        'bins_per_feature': [2,2,2,2,2],
        'bin_padding': 0.25,
        'training_idxs': training_range,
        'testing_idxs': test_range,
    }
    
    
    }
    

stock_list =["amzn_daily.csv", "appl_daily.csv","tsla_daily.csv","f_daily.csv",
             "pfe_daily.csv", "coke_daily.csv","brk_daily.csv","nee_daily.csv",
             "jnj_daily.csv","pg_daily.csv"]

stock_name_list = ['AMZN','APPL','TSLA','F','PFE','COKE','BRK','NEE','JNJ','PG']









## Code for Analysis, shouldn't have to be touched

In [3]:
results = {}

# Set Image Path for raw (Required for graph function) -- can be updated later
raw_save_path = save_path + "raw/" 
if not os.path.exists(raw_save_path):
    os.makedirs(raw_save_path)

# save path for input graphs
input_save_path = save_path + "input/" 
if not os.path.exists(input_save_path):
    os.makedirs(input_save_path)

for file_name, stock_name in zip(stock_list,stock_name_list):
      
    history_logger.info(stock_name)
    results[stock_name] = {}
    # Import File
    df_ohlcv = cleandata.NASDAQ_csv_input(file_name,import_path)

    input_graph_path_file = input_save_path + f'{stock_name}.png'
    graph_input(df_ohlcv['close'],
                stock_name,
                display_graph = output_input_graphic_option, 
                save_graphic = save_input_graphic_option,
                path_file = input_graph_path_file)
    
    for agent_name, agent_class in agent_classes.items():
        
        selected_agent = agent_class(df_ohlcv,
                                **agent_params[agent_name])

        # Output Status    
        update_text = f'{agent_name}:{stock_name} - Initialized'    
        print(update_text)


        # history Log  
        history_logger.info(update_text)
        
        results[stock_name][agent_name] = {}
                                           
        # Training and Testing Pairs  
        for (train_start_idx, train_end_idx), (test_start_idx, test_end_idx) \
            in zip(selected_agent.training_range, selected_agent.test_range):
            
            selected_agent.reset() # Wipe out any internal values
                 
            if agent_name not in no_train_agents:  ### Exclude if agent does not require training
                train_key = f'[{train_start_idx}:{train_end_idx})' 
                graph_train_key = 'TRN'+ train_key
                ### Output Status
                update_text = f'{agent_name}:{stock_name} - Training: {train_key}'    
                print(update_text)
                
                ### Train Agent
                selected_agent.train(train_start_idx,
                                     train_end_idx,
                                     initial_epsilon = 0.75, 
                                     final_epsilon = 0.1)
                ### History Logger 
                history_logger.info(update_text)
                history_logger.info(selected_agent.env.step_info[-1:])
            
            ## Testing Section 

            # Marker for Output Status, Debugging, and Return
            test_key = f'[{test_start_idx}:{test_end_idx})'
            
            # For Graph Title 
            graph_test_key = 'TST'+ test_key

            # History Log                                   
            history_logger.info(test_key)
            
            # Initializing Results Dictionary for Test Keys
            results[stock_name][agent_name][test_key] = []

            # Initializing Aggregate Trade Dataframe
            df_trade = df = pd.DataFrame(0, index = range(test_end_idx - test_start_idx), 
                                            columns=['B','H','S','Position_Count'],dtype=int)
            
            # Combine Graph Title
            if agent_name in no_train_agents:
                graph_label = f'{agent_name}$_{{{graph_test_key}}}$'
            else:
                graph_label = f'{agent_name}$_{{{graph_train_key},{graph_test_key}}}$'

            for idx in range(1, num_of_tests + 1):

                # Combine Graph Title
                if agent_name in no_train_agents:
                
                    graph_image_file = f'{agent_name}-{stock_name}-TST_{test_start_idx}_{test_end_idx}-{idx}.png'
                else:
                            
                    graph_image_file = f'{agent_name}-{stock_name}-TRN_{train_start_idx}_{train_end_idx}-TST_{test_start_idx}_{test_end_idx}-{idx}.png'
                
                raw_image_file_path = raw_save_path + graph_image_file

                # Output Status    
                update_text = f'{agent_name}:{stock_name} -  \
                    Testing: [{test_start_idx}:{test_end_idx}) - Test #: {idx}'    
                print(update_text)

   
                # Execute Test Run on Agent                    
                selected_agent.test(test_start_idx,test_end_idx)

                # Update Trade Aggregate Trade Dataframe
                ## Retrive List of Actions
                trades_list = [entry['Action'] for entry in \
                    selected_agent.env.step_info]
                ## Update DF
                df_trade = update_trade_df(df_trade,trades_list)
                
                # History Logging
                ## Name of Agent, Stock, Test indices with Test #
                history_logger.info(update_text)  
                ## Last Few Steps Information 
                history_logger.info(selected_agent.env.step_info[-5:])
                ## Trade Sequence Logging                
                trade_seq = np.array(trades_list)
                history_logger.info(trade_seq)


                test_result = agent_stock_performance(df_ohlcv['close']\
                        .iloc[test_start_idx:test_end_idx].to_numpy(),
                        trade_seq,stock_name,graph_label, 
                        display_graph=output_raw_graphic_option,
                        save_graphic=save_raw_graphic_option, 
                        path_file=raw_image_file_path)
                
                # Update Results Dic
                result_details = {metric:test_result[metric] for metric in metrics}
                results[stock_name][agent_name][test_key].append([trade_seq,result_details])

            # Aggregate Data for Test

            # Combine Graph Title
            df_trade['close'] = df_ohlcv['close'].iloc[test_start_idx:test_end_idx].values
            if agent_name in no_train_agents:
            
                agg_graph_image_file = f'{agent_name}-{stock_name}-TST_{test_start_idx}_{test_end_idx}.png'
            else:
                        
                agg_graph_image_file = f'{agent_name}-{stock_name}-TRN_{train_start_idx}_{train_end_idx}-TST_{test_start_idx}_{test_end_idx}.png'
            
            agg_image_file_path = save_path + agg_graph_image_file
            
            aggregate_trade_performance(df_trade,                                          stock_name,
                                            graph_label,
                                            num_of_tests,
                                            display_graph = output_agg_graphic_option,
                                            save_graphic = save_agg_graphic_option, 
                                            path_file = agg_image_file_path)
# Write to Test Logger
test_logger.info(results)


INFO:History_logger:AMZN


here


INFO:History_logger:Qtab_[2,2,2,2,2]_raw:AMZN - Initialized


Qtab_[2,2,2,2,2]_raw:AMZN - Initialized
Qtab_[2,2,2,2,2]_raw:AMZN - Training: [0:2000)


INFO:History_logger:Qtab_[2,2,2,2,2]_raw:AMZN - Training: [0:2000)
INFO:History_logger:[{'Step': 2000, 'idx': 1999, 'Portfolio Value': 1199287.311921501, 'Cash': 1199287.311921501, 'Stock Value': 0.0, 'Stock Holdings': 0, 'Stock Price': 176.1645, 'Last Commission Cost': 0, 'Total Commision Cost': 1051911.0853614996, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('H',), 'Action': 'H'}]
INFO:History_logger:[2000:2518)


...

INFO:History_logger:Buy_Hold_Agent:PG -                      Testing: [2000:2518) - Test #: 1
INFO:History_logger:[{'Step': 514, 'idx': 2513, 'Portfolio Value': 1764260.65, 'Cash': 5009.949999999953, 'Stock Value': 1771689.6, 'Stock Holdings': 12195, 'Stock Price': 145.28, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.95, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 515, 'idx': 2514, 'Portfolio Value': 1776699.55, 'Cash': 5009.949999999953, 'Stock Value': 1779738.3, 'Stock Holdings': 12195, 'Stock Price': 145.94, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.95, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 516, 'idx': 2515, 'Portfolio Value': 1784748.25, 'Cash': 5009.949999999953, 'Stock Value': 1781201.7, 'Stock Holdings': 12195, 'Stock Price': 146.06, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.95, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'S

Buy_Hold_Agent:PG -                      Testing: [2000:2518) - Test #: 2


INFO:History_logger:Buy_Hold_Agent:PG -                      Testing: [2000:2518) - Test #: 2
INFO:History_logger:[{'Step': 514, 'idx': 2513, 'Portfolio Value': 951404.504, 'Cash': 5058.90399999998, 'Stock Value': 953036.8, 'Stock Holdings': 6560, 'Stock Price': 145.28, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.704000000001, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 515, 'idx': 2514, 'Portfolio Value': 958095.704, 'Cash': 5058.90399999998, 'Stock Value': 957366.4, 'Stock Holdings': 6560, 'Stock Price': 145.94, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.704000000001, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 516, 'idx': 2515, 'Portfolio Value': 962425.304, 'Cash': 5058.90399999998, 'Stock Value': 958153.6, 'Stock Holdings': 6560, 'Stock Price': 146.06, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.704000000001, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H')

Buy_Hold_Agent:PG -                      Testing: [2000:2518) - Test #: 3


INFO:History_logger:Buy_Hold_Agent:PG -                      Testing: [2000:2518) - Test #: 3
INFO:History_logger:[{'Step': 514, 'idx': 2513, 'Portfolio Value': 951404.504, 'Cash': 5058.90399999998, 'Stock Value': 953036.8, 'Stock Holdings': 6560, 'Stock Price': 145.28, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.704000000001, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 515, 'idx': 2514, 'Portfolio Value': 958095.704, 'Cash': 5058.90399999998, 'Stock Value': 957366.4, 'Stock Holdings': 6560, 'Stock Price': 145.94, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.704000000001, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H'), 'Action': 'H'}, {'Step': 516, 'idx': 2515, 'Portfolio Value': 962425.304, 'Cash': 5058.90399999998, 'Stock Value': 958153.6, 'Stock Holdings': 6560, 'Stock Price': 146.06, 'Last Commission Cost': 0, 'Total Commision Cost': 4999.704000000001, 'Input': [1, 1, 1, 1, 0], 'Available Actions': ('S', 'H')

# Aggreating Test Bench Data

In [4]:
import pandas as pd

test_keys = [f"[{interval[0]}:{interval[1]})" for interval in test_range]
aggerate_results = {}
for agent in agent_list:
    aggerate_results[agent] = {}
    for stock in stock_name_list:
        aggerate_results[agent][stock] = {}
        for test in test_keys:
            aggerate_results[agent][stock][test] = {}
            values = np.empty((0,len(metrics)))
            for test_num in range(num_of_tests):
                values_array = [results[stock][agent][test][test_num][1][key] for key in metrics]
                current_values = np.array(values_array)
                values = np.vstack((values,current_values))

                means_for_metrics = np.mean(values, axis=0)
                std_for_metrics = np.std(values, axis=0)

            
            
            for idx,metric in enumerate(metrics):
                aggerate_results[agent][stock][test][metric] = (means_for_metrics[idx],std_for_metrics[idx])

display(aggerate_results)

summarized_aggerate_results = {}

for metric in metrics:
    model_list = []
    dataset_name = []
    scores = []
    for agent in aggerate_results.keys():
        
        model_list.append(agent)
        score_list = []
        for stock in aggerate_results[agent].keys():
            for test in aggerate_results[agent][stock].keys():
                run_name = stock + "-" + test
                if run_name not in dataset_name:
                    dataset_name.append(run_name)
                score = aggerate_results[agent][stock][test][metric][0]
                score_list.append(np.round(score,2))
        scores.append(score_list)

    score_array = np.array(scores).T

    df = pd.DataFrame(score_array,columns=model_list)
    df['dataset'] = dataset_name

    column_order = ['dataset'] + [col for col in df.columns if col != 'dataset']
    df = df[column_order]

    model_means = list(zip(model_list,df[model_list].mean()))

    summarized_aggerate_results[metric] = (df, model_means)


display(summarized_aggerate_results)



{'Qtab_[2,2,2,2,2]_raw': {'AMZN': {'[2000:2518)': {'n_trades': (22.666666666666668,
     3.39934634239519),
    'win_percentage': (64.85042735042735, 1.7424686357158186),
    'cumulative_return': (1.0850805080079786, 0.10294313016397799),
    'sortino': (4.315617166444737, 5.0683180049035625),
    'max_drawdown': (-9.916754916937114, 2.900497058619771),
    'sharpe': (2.2488055397734024, 2.3934064662677437)}},
  'APPL': {'[2000:2518)': {'n_trades': (22.0, 2.8284271247461903),
    'win_percentage': (52.56410256410256, 12.772988840458394),
    'cumulative_return': (1.0378060961645628, 0.11055912130486603),
    'sortino': (3.8827640022233347, 5.67145323740303),
    'max_drawdown': (-8.284938455711432, 5.628881986018332),
    'sharpe': (1.7657950971040088, 3.3782059062230716)}},
  'TSLA': {'[2000:2518)': {'n_trades': (17.0, 2.8284271247461903),
    'win_percentage': (36.507936507936506, 11.77602348634375),
    'cumulative_return': (1.0807872460365713, 0.6301716993545676),
    'sortino': (4

{'n_trades': (            dataset  Qtab_[2,2,2,2,2]_raw  Qtab_[10,10,10,10,10]_raw  \
  0  AMZN-[2000:2518)                 22.67                      23.00   
  1  APPL-[2000:2518)                 22.00                      27.00   
  2  TSLA-[2000:2518)                 17.00                      29.33   
  3     F-[2000:2518)                 20.00                      36.00   
  4   PFE-[2000:2518)                 20.33                      20.67   
  5  COKE-[2000:2518)                 25.33                      24.67   
  6   BRK-[2000:2518)                 22.67                      23.33   
  7   NEE-[2000:2518)                 24.00                      25.67   
  8   JNJ-[2000:2518)                 26.33                      25.33   
  9    PG-[2000:2518)                 27.33                      27.00   
  
     Qtab_[10,10,10,10,10]_diff  Qtab_[10,10,10,10,10]_ma  \
  0                       22.67                     27.33   
  1                       25.67                  

# Significance

In [5]:
import prob_evaluate

for metric, rank_type in list(zip(metrics,rank_order)):
    test = prob_evaluate.generate_rank_array_from_dataframe(summarized_aggerate_results[metric][0],
                                                            model_list,equal_rank_behav="mean",rank_order=rank_type)
    
    
    model_compare_logger.info(f'\n\n\n-------{metric}--------\nFriedman Test:')
    model_compare_logger.info(summarized_aggerate_results[metric][0])
    model_compare_logger.info(test.T)
    stat, critical_f_value, reject_null_hypo = prob_evaluate.iman_davenport_test(test,0.95)

    model_compare_logger.info(f'f-score: {stat:.3f}\nf-critical: {critical_f_value:.3f}\nSignificant: {reject_null_hypo}')

    results1 = prob_evaluate.nemenyi_test(test,0.95,model_list)
    model_compare_logger.info(f'Nemenyi Test Resutls:\n{results1}')

INFO:Model_compare_logger:


-------n_trades--------
Friedman Test:


INFO:Model_compare_logger:            dataset  Qtab_[2,2,2,2,2]_raw  Qtab_[10,10,10,10,10]_raw  \
0  AMZN-[2000:2518)                 22.67                      23.00   
1  APPL-[2000:2518)                 22.00                      27.00   
2  TSLA-[2000:2518)                 17.00                      29.33   
3     F-[2000:2518)                 20.00                      36.00   
4   PFE-[2000:2518)                 20.33                      20.67   
5  COKE-[2000:2518)                 25.33                      24.67   
6   BRK-[2000:2518)                 22.67                      23.33   
7   NEE-[2000:2518)                 24.00                      25.67   
8   JNJ-[2000:2518)                 26.33                      25.33   
9    PG-[2000:2518)                 27.33                      27.00   

   Qtab_[10,10,10,10,10]_diff  Qtab_[10,10,10,10,10]_ma  \
0                       22.67                     27.33   
1                       25.67                     22.33   
2   